In [55]:
import os
import time
import random
import csv
import json
from tqdm.auto import tqdm

import requests
from bs4 import BeautifulSoup

In [56]:
start_page = 1
end_page = 172 
base_url = 'https://www.penize.cz/poradna/dane'
csv_filename = '../data/qa_penize.csv'
csv_header = ['question_date', 'question_title', 'question_text', 'QA_link', 'answer_date', 'answer_text']

In [57]:
file_exists = os.path.isfile(csv_filename)
mode = 'a' if file_exists else 'w'
with open (csv_filename, mode, newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    if not file_exists:
        writer.writerow(csv_header)
    for page in tqdm(range(start_page, end_page + 1)):
        url = f'{base_url}?pagenumber={page}'
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error on page {page}: HTTP Status Code {response.status_code}")
            continue
        soup = BeautifulSoup(response.text, 'html.parser')
        h4_tags = soup.find_all('h4', class_='title')
        for h4_tag in h4_tags:
            a_tag = h4_tag.find('a')
            QA_link = a_tag['href']
            QA_response = requests.get(base_url + QA_link)
            if QA_response.status_code != 200:
                print(f"Error on page {page}: HTTP Status Code {QA_response.status_code}")
                continue
            QA_soup = BeautifulSoup(QA_response.text, 'html.parser')
            script_tag = QA_soup.find("script", type="application/ld+json")
            json_content = script_tag.string.strip()
            json_data = json.loads(json_content, strict=False)
            question_date = json_data["mainEntity"]["datePublished"]
            question_title = json_data["mainEntity"]["name"]
            question_text = json_data["mainEntity"]["text"].replace('\n', ' ').replace('\r', ' ').replace('  ', ' ')
            answer_date = json_data["mainEntity"]["acceptedAnswer"]["datePublished"]
            answer_text = json_data["mainEntity"]["acceptedAnswer"]["text"].replace('\n', ' ').replace('\r', ' ').replace('  ', ' ')
            writer.writerow([question_date, question_title, question_text, QA_link, answer_date, answer_text])
        time.sleep(random.uniform(0.5, 1.5))

  0%|          | 0/172 [00:00<?, ?it/s]